In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/data.csv')

In [3]:
df.head()

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad


In [4]:
df.values

array([['diaryofagameaddict.com', 'bad'],
       ['espdesign.com.au', 'bad'],
       ['iamagameaddict.com', 'bad'],
       ...,
       ['apple-iclods.org/', 'bad'],
       ['apple-uptoday.org/', 'bad'],
       ['apple-search.info', 'bad']], dtype=object)

In [5]:
from features import URLFeatures

from pprint import pprint 
import re
def get_meta(url):


    def get_length_of_primary(url):
        return len(url.split('//')[-1].split('/')[0])

    def get_number_of_dots(url):
        return len(url.split('.')) - 1

    def check_if_url_has_ip(url):
        return True if re.match(r'http://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}/.*', url) else False

    def get_prop(property_list):
        if isinstance(property_list, list):
            return property_list[0], len(property_list)
        else:
            return property_list, 1

    def get_words_from_URL(url):
        words = re.compile(r'[\:/?=\-&.]+',re.UNICODE).split(url)
        sortedwords = sorted(words, key=len)
        average = sum([len(wrd) for wrd in words])/len(words)
        return len(sortedwords[-1]), len(sortedwords[0]), average



    obj = URLFeatures(url)
    extracted = obj.to_dict()
    extracted['actual_nameservers'], extracted['actual_nameservers_count'] = get_prop(extracted['actual_nameservers']) 
    extracted['whois_nameservers'], extracted['whois_nameservers_count'] = get_prop(extracted['whois_nameservers']) 
    extracted['emails'], extracted['emails_count'] = get_prop(extracted['emails']) 
    extracted['lifetime'] = (pd.to_datetime(extracted['expiration_date']) - pd.to_datetime(extracted['creation_date'])).days
    extracted['length'] = len(url)
    extracted['length_of_primary'] = get_length_of_primary(url)
    extracted['number_of_dots'] = get_number_of_dots(url)
    extracted['has_ip'] = check_if_url_has_ip(url)
    extracted['longest_words'], extracted['shortest_words'], extracted['avg_words'] = get_words_from_URL(url)
    return pd.DataFrame(extracted, index=[0])



In [6]:
from tqdm import tqdm_notebook as tq
%matplotlib inline  
res = pd.DataFrame()
fail = pd.DataFrame()
def process_row(row):
    try:
        r = get_meta(row[0])
        r[0], r[1] = row[0], row[1]
        return r
    except:
        r = pd.DataFrame()
        r[0], r[1] = row[0], row[1]
        return r
        
    


In [ ]:
import multiprocessing

p = multiprocessing.Pool(50)
for x in p.imap_unordered(process_row, df.values):
    if len(x.columns)>2:
        try:
            res = pd.read_csv('dataset_new.csv')
            if len(res%15000 == 0):
                res.to_csv('dataset_new_ckpt.csv', index=False, encoding='utf-8')
        except:
            pass
        res = res.append(x, ignore_index=True)
        
        res.to_csv('dataset_new.csv', index=False, encoding='utf-8')
    else:
        try:
            fail = pd.read_csv('failed_new.csv')
        except:
            pass
        fail = fail.append(x, ignore_index=True)
        fail.to_csv('failed_new.csv', index=False, encoding='utf-8')

unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/tldextract/.tld_set'
unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/tldextract/.tld_set'
unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/tldextract/.tld_set'
unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/tldextract/.tld_set'
unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.5/dist-packages/tldextract/.tld_set'
unable to cache TLDs in file /usr/local/lib/python3.5/dist-packages/tldextract/.tld_set: [Errno 13] 

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os
from sklearn.linear_model import LogisticRegression


def getTokens(input):
    tokensBySlash = str(input.encode('utf-8')).split('/')#get tokens after splitting by slash
    allTokens = []
    for i in tokensBySlash:
        tokens = str(i).split('-')#get tokens after splitting by dash
        tokensByDot = []
        for j in range(0,len(tokens)):
            tempTokens = str(tokens[j]).split('.')#get tokens after splitting by dot
            tokensByDot = tokensByDot + tempTokens
        allTokens = allTokens + tokens + tokensByDot
    allTokens = list(set(allTokens))#remove redundant tokens
    if 'com' in allTokens:
        allTokens.remove('com')#removing .com since it occurs a lot of times and it should not be included in our features
    return allTokens


allurls = 'data/data.csv'#path to our all urls file
allurlscsv = pd.read_csv(allurls,',',error_bad_lines=False)	#reading file
allurlsdata = pd.DataFrame(allurlscsv)#converting to a dataframe

allurlsdata = np.array(allurlsdata)#converting it into an array
random.shuffle(allurlsdata)#shuffling

y = [d[1] for d in allurlsdata]#all labels 
corpus = [d[0] for d in allurlsdata]#all urls corresponding to a label (either good or bad)
vectorizer = TfidfVectorizer(tokenizer=getTokens)#get a vector for each url but use our customized tokenizer
X = vectorizer.fit_transform(corpus)#get the X vector

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)#split into training and testing set 80/20 ratio

lgs = LogisticRegression()#using logistic regression
lgs.fit(X_train, y_train)
print(lgs.score(X_test, y_test))#pring the score. It comes out to be 98%

0.9838393207520245


<1x395281 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>